In [23]:
"""
import pulp
pulp.listSolvers(onlyAvailable=True)
"""

['PULP_CBC_CMD']

In [24]:
from pathlib import Path
instances_folder = Path.cwd().parent / 'instances'

file_path = instances_folder / 'your_file.txt'


In [25]:
from pathlib import Path

instances_folder = Path.cwd().parent / 'instances'
filename = 'inst01.dat'
file_path = instances_folder / filename

def read_integers_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        numbers = list(map(int, line.split()))
        data.append(numbers)
    
    return data

data = read_integers_from_file(file_path)
couriers = data[0][0]
items = data[1][0]
l = data[2]
s = data[3]
D = data[4:]
nodes = items+1

In [26]:
def reorder_distance_matrix(D): #Just to chech if i missed something with classical formulation
    last_row = D[-1]
    last_column = [row[-1] for row in D]
    
    D_reduced = [row[:-1] for row in D[:-1]]
    
    new_matrix = []
    
    new_matrix.append([last_row[-1]] + last_row[:-1])
    
    for i in range(len(D_reduced)):
        new_matrix.append([last_column[i]] + D_reduced[i])
    
    return new_matrix

D = reorder_distance_matrix(D)
D


[[0, 2, 3, 4, 3, 4, 4],
 [2, 0, 3, 4, 5, 6, 6],
 [3, 3, 0, 1, 4, 5, 7],
 [4, 4, 1, 0, 5, 6, 6],
 [2, 4, 4, 5, 0, 3, 3],
 [4, 6, 7, 8, 3, 0, 2],
 [4, 6, 7, 8, 3, 2, 0]]

In [27]:
from pulp import LpProblem, LpVariable, lpSum, LpBinary, LpMinimize,LpContinuous,LpInteger,PULP_CBC_CMD,getSolver

prob = LpProblem("Vehicle_Routing_Problem", LpMinimize)
X = LpVariable.dicts("X", (range(nodes), range(nodes), range(couriers)), cat=LpBinary)

# Define the maximum distance variable Z
Z = LpVariable("Z", lowBound=0)

# MTZ Constraints
u = LpVariable.dicts("u", range(nodes), lowBound=0, upBound=nodes-1, cat=LpInteger)


In [28]:
# No self loop
for k in range(couriers): 
    for i in range(nodes):
         prob += X[i][i][k] == 0

#1 Vehicle leaves node that it enters
for k in range(couriers):
    for j in range(nodes): 
        prob += lpSum(X[i][j][k] for i in range(nodes)) == lpSum(X[j][i][k] for i in range(nodes))

#2 Ensure that every node is entered once
for j in range(1,nodes): #skipping depot = 0
    prob += lpSum(X[i][j][k] for i in range(nodes) for k in range(couriers)) == 1

#3 Ensure every vehicle leaves the depot
for k in range(couriers):
    prob += lpSum(X[0][j][k] for j in range(1,nodes)) == 1

#Capacity constraint
for k in range(couriers):
    prob += lpSum(X[i][j][k] * s[j-1] for i in range(nodes) for j in range(1,nodes)) <= l[k]  #s[j-1] because we shifted depot at position 0

# Add constraints for Z
for k in range(couriers):
    prob += lpSum(D[i][j] * X[i][j][k] for i in range(nodes) for j in range(nodes)) <= Z

# Eliminating subtours
for k in range(couriers):
    for i in range(1, nodes):
        for j in range(1, nodes):
            prob += u[i] - u[j] + (nodes-1) * X[i][j][k] <= nodes-2

prob += Z
prob.solve()

1

In [29]:
print("Optimal value for Z:", Z.varValue)
for k in range(couriers):
    print(f"Courier {k} route:")
    for i in range(items + 1):
        for j in range(items + 1):
            if X[i][j][k].varValue == 1:
                print(f"  {i} -> {j}")

Optimal value for Z: 14.0
Courier 0 route:
  0 -> 1
  1 -> 3
  3 -> 4
  4 -> 0
Courier 1 route:
  0 -> 2
  2 -> 5
  5 -> 6
  6 -> 0


In [33]:
def test_instance(file_path, filename):
    data = read_integers_from_file(file_path)
    couriers = data[0][0]
    items = data[1][0]
    l = data[2]
    s = data[3]
    D = data[4:]
    nodes = items+1
    D = reorder_distance_matrix(D)

    prob = LpProblem("Vehicle_Routing_Problem", LpMinimize)
    X = LpVariable.dicts("X", (range(nodes), range(nodes), range(couriers)), cat=LpBinary)

    # Define the maximum distance variable Z
    Z = LpVariable("Z", lowBound=0)
    # No self loop
    for k in range(couriers): 
        for i in range(nodes):
            prob += X[i][i][k] == 0

    #1 Vehicle leaves node that it enters
    for k in range(couriers):
        for j in range(nodes): 
            prob += lpSum(X[i][j][k] for i in range(nodes)) == lpSum(X[j][i][k] for i in range(nodes))

    #2 Ensure that every node is entered once
    for j in range(1,nodes): #skipping depot = 0
        prob += lpSum(X[i][j][k] for i in range(nodes) for k in range(couriers)) == 1

    #3 Ensure every vehicle leaves the depot
    for k in range(couriers):
        prob += lpSum(X[0][j][k] for j in range(1,nodes)) == 1

    #Capacity constraint
    for k in range(couriers):
        prob += lpSum(X[i][j][k] * s[j-1] for i in range(nodes) for j in range(1,nodes)) <= l[k]  #s[j-1] because we shifted depot at position 0

    # Add constraints for Z
    for k in range(couriers):
        prob += lpSum(D[i][j] * X[i][j][k] for i in range(nodes) for j in range(nodes)) <= Z

    # MTZ Constraints
    u = LpVariable.dicts("u", range(nodes), lowBound=0, upBound=nodes-1, cat=LpContinuous)

    # Ensure valid flow in the network
    for k in range(couriers):
        for i in range(1, nodes):
            for j in range(1, nodes):
    #            if i != j:
                prob += u[i] - u[j] + (nodes-1) * X[i][j][k] <= nodes-2
    prob += Z
    solver = getSolver('PULP_CBC_CMD', timeLimit=10)

    prob.solve(solver)

    # Print results
    print(f"Status: {pulp.LpStatus[prob.status]}")
    print(f"Optimal value for {filename}:", Z.varValue)

In [9]:
for i in range(1, 22):
    filename = f'inst{str(i).zfill(2)}.dat'
    file_path = instances_folder / filename
    test_instance(file_path,filename)

Status: Optimal
Optimal value for instances/inst01.dat: 14.0
Status: Optimal
Optimal value for instances/inst02.dat: 226.0
Status: Optimal
Optimal value for instances/inst03.dat: 12.0
Status: Optimal
Optimal value for instances/inst04.dat: 220.0
Status: Optimal
Optimal value for instances/inst05.dat: 206.0
Status: Optimal
Optimal value for instances/inst06.dat: 322.0
Status: Optimal
Optimal value for instances/inst07.dat: 168.0
Status: Optimal
Optimal value for instances/inst08.dat: 186.0
Status: Optimal
Optimal value for instances/inst09.dat: 436.0
Status: Optimal
Optimal value for instances/inst10.dat: 244.0
Status: Not Solved
Optimal value for instances/inst11.dat: 103.64608
Status: Not Solved
Optimal value for instances/inst12.dat: 82.899919
Status: Optimal
Optimal value for instances/inst13.dat: 540.0
